In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, \
                            recall_score, f1_score, log_loss

from tqdm.notebook import tqdm

from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

import joblib


RAND=42
N_FOLDS = 5

In [2]:
def get_metrics(y_test, y_pred, y_score, name = "Default"):
    """Метрики для задачи классификации"""
    df_metrics = pd.DataFrame()

    df_metrics['model'] = [name]
    df_metrics['Accuracy'] = accuracy_score(y_test, y_pred)
    df_metrics['ROC_AUC'] = roc_auc_score(y_test, y_score[:, 1])
    df_metrics['Precision'] = precision_score(y_test, y_pred)
    df_metrics['Recall'] = recall_score(y_test, y_pred)
    df_metrics['f1'] = f1_score(y_test, y_pred)
    df_metrics['Logloss'] = log_loss(y_test, y_score)

    return df_metrics

# Feature engineering

In [3]:
filepath = "../data/processed/train_data_proc.pq"
df = pd.read_parquet(filepath)
df[:5]

,rn,pre_since_opened,pre_since_confirmed,pre_pterm,pre_fterm,pre_till_pclose,pre_till_fclose,pre_loans_credit_limit,pre_loans_next_pay_summ,pre_loans_outstanding,...,enc_paym_21_5,enc_paym_22_5,enc_paym_23_5,enc_paym_24_5,enc_loans_account_holder_type_5,enc_loans_credit_status_5,enc_loans_credit_type_5,enc_loans_account_cur_5,pclose_flag_5,fclose_flag_5
id,,,,,,,,,,,,,,,,,,,,,
0,10,7,9,2,10,8,8,16,4,2,...,3.0,3.0,3.0,4.0,1.0,2.0,3.0,1.0,0.0,1.0
1,14,12,6,1,8,14,11,11,2,3,...,0.0,0.0,0.0,1.0,1.0,2.0,3.0,1.0,0.0,0.0
2,3,12,9,4,8,1,11,1,1,4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,15,12,9,2,10,8,8,17,1,2,...,3.0,3.0,3.0,4.0,1.0,3.0,4.0,1.0,0.0,0.0
4,1,12,9,4,8,1,11,12,1,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
df.shape

(237887, 292)

In [5]:
X = df.drop(columns=["flag"])
Y = df["flag"]

In [6]:
mms = MinMaxScaler()
X_scaled = mms.fit_transform(X)

In [7]:
x_train, x_test, y_train, y_test = train_test_split(X_scaled, Y, test_size=0.2, 
                                                    stratify=Y, 
                                                    random_state=RAND)

# Baseline models

In [11]:
# Создаем и обучаем бейслайн LightGBM
model = LGBMClassifier(class_weight='balanced', random_state=RAND)
model.fit(x_train, y_train)

LGBMClassifier(class_weight='balanced', random_state=42)

In [12]:
y_pred_tr = model.predict(x_train)
y_score_tr = model.predict_proba(x_train)

y_pred = model.predict(x_test)
y_score = model.predict_proba(x_test)

In [13]:
res = get_metrics(y_train, y_pred_tr, y_score_tr, name="lgbm_train")
res = res.append(get_metrics(y_test, y_pred, y_score, name="lgbm_test"))

/tmp/ipykernel_37876/158785131.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res = res.append(get_metrics(y_test, y_pred, y_score, name="lgbm_test"))


In [14]:
res

,model,Accuracy,ROC_AUC,Precision,Recall,f1,Logloss
0,lgbm_train,0.745855,0.855194,0.091475,0.807823,0.164340,0.508421
0,lgbm_test,0.734860,0.749605,0.071062,0.627020,0.127657,0.521550


In [15]:
ratio = y_train[y_train==0].shape[0] / y_train[y_train==1].shape[0]

# Создаем и обучаем бейслайн Catboost
model = CatBoostClassifier(scale_pos_weight=ratio, random_state=RAND, verbose=0)
model.fit(x_train, y_train)

In [16]:
y_pred_tr = model.predict(x_train)
y_score_tr = model.predict_proba(x_train)

y_pred = model.predict(x_test)
y_score = model.predict_proba(x_test)

In [17]:
res = res.append(get_metrics(y_train, y_pred_tr, y_score_tr, name="catboost_train"))
res = res.append(get_metrics(y_test, y_pred, y_score, name="catboost_test"))

/tmp/ipykernel_37876/1045409491.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res = res.append(get_metrics(y_train, y_pred_tr, y_score_tr, name="catboost_train"))
/tmp/ipykernel_37876/1045409491.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res = res.append(get_metrics(y_test, y_pred, y_score, name="catboost_test"))


In [18]:
res

,model,Accuracy,ROC_AUC,Precision,Recall,f1,Logloss
0,lgbm_train,0.745855,0.855194,0.091475,0.807823,0.164340,0.508421
0,lgbm_test,0.734860,0.749605,0.071062,0.627020,0.127657,0.521550
0,catboost_train,0.877820,0.971701,0.197413,0.962179,0.327610,0.318028
0,catboost_test,0.846960,0.712020,0.081448,0.383969,0.134389,0.361382


In [19]:
# Создаем и обучаем линейную регрессию
model = LogisticRegression(class_weight='balanced', random_state=RAND)
model.fit(x_train, y_train)

/home/sergey/virtual_env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(class_weight='balanced', random_state=42)

In [20]:
y_pred_tr = model.predict(x_train)
y_score_tr = model.predict_proba(x_train)

y_pred = model.predict(x_test)
y_score = model.predict_proba(x_test)

In [21]:
res = res.append(get_metrics(y_train, y_pred_tr, y_score_tr, "LG_train"))
res = res.append(get_metrics(y_test, y_pred, y_score, "LG_test"))
res

/tmp/ipykernel_37876/3198375833.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res = res.append(get_metrics(y_train, y_pred_tr, y_score_tr, "LG_train"))
/tmp/ipykernel_37876/3198375833.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res = res.append(get_metrics(y_test, y_pred, y_score, "LG_test"))


,model,Accuracy,ROC_AUC,Precision,Recall,f1,Logloss
0,lgbm_train,0.745855,0.855194,0.091475,0.807823,0.164340,0.508421
0,lgbm_test,0.734860,0.749605,0.071062,0.627020,0.127657,0.521550
0,catboost_train,0.877820,0.971701,0.197413,0.962179,0.327610,0.318028
0,catboost_test,0.846960,0.712020,0.081448,0.383969,0.134389,0.361382
0,LG_train,0.671110,0.748030,0.063672,0.702764,0.116766,0.598165
0,LG_test,0.667540,0.733413,0.061183,0.679379,0.112256,0.603678


**Выводы**  
Среди выбранных baseline-моделей catboost показал большое переобучение, что можно нивилировать, подобрав гиперпараметры; lightgbm показал меньше переобучение и большое значение ROC-AUC на валидации; логистическая регрессия показала слишком низкие метрики и на трейне и на валидации, что может говорить о том, что линейная модель не подходит для данной задачи.
  
Так как в нашем случае матрица объект-признаки имеет большую размерность, lightgbm является более подходящей моделью для обучения на стационарном устройстве, так как сам алгоритм работает быстрее.

# Tune

In [8]:
import optuna
import scipy.stats as stats

In [12]:
def objective_gpt(trial: optuna.Trial, x, y):
    params = {
        'n_estimators': trial.suggest_categorical('n_estimators', [1000]),
#         'learning_rate': trial.suggest_float('learning_rate', 0.001, 3),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.02143869557000772]),
        'max_bin': trial.suggest_int('max_bin', 10, 500),
        'num_leaves': trial.suggest_int('num_leaves', 10, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'min_child_samples': trial.suggest_int('min_child_samples', 100, 70000, step=100),
        'lambda_l1': trial.suggest_int('lambda_l1', 0, 100),
        'lambda_l2': trial.suggest_int('lambda_l2', 0, 100),
        'min_split_gain': trial.suggest_loguniform('min_split_gain', 1e-4, 1e-1),
        'objective': trial.suggest_categorical('objective', ['binary']),
        'metric': trial.suggest_categorical('metric', ['auc']),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.2, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.2, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'random_state': trial.suggest_categorical('random_state', [RAND]),
    }

    cv_pred = np.empty(N_FOLDS)
    cv = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=RAND)

    for fold, (train_idx, test_idx) in enumerate(cv.split(x, y)):
        x_train_, x_val_ = x[train_idx, :], x[test_idx, :]
        y_train_, y_val_ = y[train_idx], y[test_idx]

#         ratio = y_train_[y_train_ == 0].shape[0] / \
#             y_train_[y_train_ == 1].shape[0]

        pruning = optuna.integration.LightGBMPruningCallback(trial, 'auc')

        model = LGBMClassifier(
            class_weight="balanced",
            n_jobs=-1,
            **params
        )
        model.fit(x_train_, y_train_,
                  eval_metric='auc',
                  eval_set=[(x_val_, y_val_)],
                  early_stopping_rounds=100,
                  callbacks=[pruning],
                  verbose=0)

        y_pred = model.predict(x_val_)
        y_proba = model.predict_proba(x_val_)[:, 1]

        cv_pred[fold] = roc_auc_score(y_val_, y_proba)
    return (np.mean(cv_pred))

In [13]:
func = lambda trial: objective_gpt(trial, x_train, y_train.values)

study = optuna.create_study(direction="maximize")
study.optimize(func, n_trials=10, show_progress_bar=True, n_jobs=4)

[I 2023-06-03 22:40:21,375] A new study created in memory with name: no-name-3fefaafe-5fc4-40f0-89de-94c0dda38c88
/home/sergey/virtual_env/lib/python3.10/site-packages/optuna/progress_bar.py:56: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


  0%|          | 0/10 [00:00<?, ?it/s]

/tmp/ipykernel_86644/1078436011.py:12: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'min_split_gain': trial.suggest_loguniform('min_split_gain', 1e-4, 1e-1),
/tmp/ipykernel_86644/1078436011.py:15: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'feature_fraction': trial.suggest_uniform('feature_fraction', 0.2, 1.0),
/tmp/ipykernel_86644/1078436011.py:16: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.2, 1.0),
/tmp/ipyk

[LightGBM] [Warning] lambda_l1 is set=95, reg_alpha=0.0 will be ignored. Current value: lambda_l1=95
[LightGBM] [Warning] bagging_fraction is set=0.9566302906697002, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9566302906697002
[LightGBM] [Warning] feature_fraction is set=0.9764786505124872, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9764786505124872
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] lambda_l2 is set=91, reg_lambda=0.0 will be ignored. Current value: lambda_l2=91
[LightGBM] [Warning] lambda_l1 is set=70, reg_alpha=0.0 will be ignored. Current value: lambda_l1=70
[LightGBM] [Warning] bagging_fraction is set=0.9005339426029244, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9005339426029244
[LightGBM] [Warning] feature_fraction is set=0.2100025371040685, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.210

/home/sergey/virtual_env/lib/python3.10/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/sergey/virtual_env/lib/python3.10/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] lambda_l1 is set=49, reg_alpha=0.0 will be ignored. Current value: lambda_l1=49
[LightGBM] [Warning] bagging_fraction is set=0.562334832260847, subsample=1.0 will be ignored. Current value: bagging_fraction=0.562334832260847
[LightGBM] [Warning] feature_fraction is set=0.28349557115473223, colsample_bytree=1.0 will be i

limit_output extension: Maximum message size of 10000 exceeded with 10270 characters

In [15]:
study.optimize(func, n_trials=20, show_progress_bar=True, n_jobs=4)

/home/sergey/virtual_env/lib/python3.10/site-packages/optuna/progress_bar.py:56: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


  0%|          | 0/20 [00:00<?, ?it/s]

/tmp/ipykernel_86644/1078436011.py:12: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'min_split_gain': trial.suggest_loguniform('min_split_gain', 1e-4, 1e-1),
/tmp/ipykernel_86644/1078436011.py:15: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'feature_fraction': trial.suggest_uniform('feature_fraction', 0.2, 1.0),
/tmp/ipykernel_86644/1078436011.py:16: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.2, 1.0),
/tmp/ipyk

[LightGBM] [Warning] lambda_l1 is set=5, reg_alpha=0.0 will be ignored. Current value: lambda_l1=5
[LightGBM] [Warning] bagging_fraction is set=0.6864657848899626, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6864657848899626
[LightGBM] [Warning] feature_fraction is set=0.4490945772777042, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4490945772777042
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lambda_l2 is set=62, reg_lambda=0.0 will be ignored. Current value: lambda_l2=62
[LightGBM] [Warning] lambda_l1 is set=0, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0
[LightGBM] [Warning] bagging_fraction is set=0.6510107202708055, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6510107202708055
[LightGBM] [Warning] feature_fraction is set=0.47524550653290504, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.475245

/tmp/ipykernel_86644/1078436011.py:12: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'min_split_gain': trial.suggest_loguniform('min_split_gain', 1e-4, 1e-1),
/tmp/ipykernel_86644/1078436011.py:15: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'feature_fraction': trial.suggest_uniform('feature_fraction', 0.2, 1.0),
/tmp/ipykernel_86644/1078436011.py:16: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.2, 1.0),
/home/ser

limit_output extension: Maximum message size of 10000 exceeded with 10276 characters

In [16]:
study.best_params

{'n_estimators': 1000,
 'learning_rate': 0.02143869557000772,
 'max_bin': 398,
 'num_leaves': 741,
 'max_depth': 6,
 'min_child_samples': 1400,
 'lambda_l1': 28,
 'lambda_l2': 78,
 'min_split_gain': 0.009700550235633551,
 'objective': 'binary',
 'metric': 'auc',
 'feature_fraction': 0.7958098010040767,
 'bagging_fraction': 0.7688979125683639,
 'bagging_freq': 3,
 'random_state': 42}

In [17]:
# Посчитаем микро- и макро- усреднение roc-auc 
cv = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=RAND)
roc_aucs = []
y_pred_hold = []
y_score_hold = []

for fold, (train_idx, test_idx) in enumerate(cv.split(x_train, y_train)):
    x_train_, x_val_ = x_train[train_idx, :], x_train[test_idx, :]
    y_train_, y_val_ = y_train.values[train_idx], y_train.values[test_idx]

#     ratio = y_train_[y_train_ == 0].shape[0] / \
#         y_train_[y_train_ == 1].shape[0]

    model = LGBMClassifier(**study.best_params, class_weight='balanced', n_jobs=-1)
    model.fit(x_train_, y_train_,
              eval_metric='auc',
              eval_set=[(x_val_, y_val_)],
              early_stopping_rounds=100,
              verbose=0)

    y_score = model.predict_proba(x_val_)

    roc_aucs.append(roc_auc_score(y_val_, y_score[:, 1]))

    y_pred_hold.append(model.predict(x_test))
    y_score_hold.append(model.predict_proba(x_test))

y_pred = stats.mode(np.column_stack(y_pred_hold), axis=1)[0]
y_score = np.column_stack([np.mean(np.column_stack(y_score_hold)[:, ::2], axis=1),
                           np.mean(np.column_stack(y_score_hold)[:, 1::2], axis=1)])

/home/sergey/virtual_env/lib/python3.10/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/sergey/virtual_env/lib/python3.10/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] lambda_l1 is set=28, reg_alpha=0.0 will be ignored. Current value: lambda_l1=28
[LightGBM] [Warning] bagging_fraction is set=0.7688979125683639, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7688979125683639
[LightGBM] [Warning] feature_fraction is set=0.7958098010040767, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7958098010040767
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] lambda_l2 is set=78, reg_lambda=0.0 will be ignored. Current value: lambda_l2=78


/home/sergey/virtual_env/lib/python3.10/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/sergey/virtual_env/lib/python3.10/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] lambda_l1 is set=28, reg_alpha=0.0 will be ignored. Current value: lambda_l1=28
[LightGBM] [Warning] bagging_fraction is set=0.7688979125683639, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7688979125683639
[LightGBM] [Warning] feature_fraction is set=0.7958098010040767, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7958098010040767
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] lambda_l2 is set=78, reg_lambda=0.0 will be ignored. Current value: lambda_l2=78


/home/sergey/virtual_env/lib/python3.10/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/sergey/virtual_env/lib/python3.10/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


...

/home/sergey/virtual_env/lib/python3.10/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/sergey/virtual_env/lib/python3.10/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] lambda_l1 is set=28, reg_alpha=0.0 will be ignored. Current value: lambda_l1=28
[LightGBM] [Warning] bagging_fraction is set=0.7688979125683639, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7688979125683639
[LightGBM] [Warning] feature_fraction is set=0.7958098010040767, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7958098010040767
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] lambda_l2 is set=78, reg_lambda=0.0 will be ignored. Current value: lambda_l2=78


/home/sergey/virtual_env/lib/python3.10/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/sergey/virtual_env/lib/python3.10/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] lambda_l1 is set=28, reg_alpha=0.0 will be ignored. Current value: lambda_l1=28
[LightGBM] [Warning] bagging_fraction is set=0.7688979125683639, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7688979125683639
[LightGBM] [Warning] feature_fraction is set=0.7958098010040767, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7958098010040767
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] lambda_l2 is set=78, reg_lambda=0.0 will be ignored. Current value: lambda_l2=78


/tmp/ipykernel_86644/2488522396.py:28: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  y_pred = stats.mode(np.column_stack(y_pred_hold), axis=1)[0]


In [18]:
print(np.mean(roc_aucs))
print(roc_auc_score(y_test, y_score[:, 1]))

0.7612821237125502
0.7563662543239633


In [19]:
params = study.best_params

joblib.dump(params, "../data/params.joblib")

['../data/params.joblib']

# Out-of-core training

In [3]:
from scrypt import out_of_core_train

In [4]:
train_path = "../data/processed/"
target_path = "../data/raw/train_target.csv"

params = joblib.load("../data/params.joblib")

In [5]:
model = out_of_core_train(train_path, target_path, params=params)

[LightGBM] [Warning] lambda_l1 is set=28, reg_alpha=0.0 will be ignored. Current value: lambda_l1=28
[LightGBM] [Warning] bagging_fraction is set=0.7688979125683639, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7688979125683639
[LightGBM] [Warning] feature_fraction is set=0.7958098010040767, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7958098010040767
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] lambda_l2 is set=78, reg_lambda=0.0 will be ignored. Current value: lambda_l2=78


  0%|          | 0/11 [00:00<?, ?it/s]

In [6]:
# Создаем словарь с моделью и MinMaxScaler'ом
models = {}

models['lightgbm'] = model

In [7]:
# Сохраняем словарь с моделями в joblib файл
models_path = '../data/models.joblib'
joblib.dump(models, models_path)

['../data/models.joblib']